In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import requests 
import io
import seaborn as sns

In [ ]:
orig_members = pd.read_csv('/content/drive/MyDrive/text gcn/Hackforum data/CSV Data IronMarch/data_csv/orig_members.csv')

create userID map (unknown range to fixed range [1 to 753]

In [ ]:
sorted_member_id = orig_members['member_id'].sort_values()
member_id_map ={}
start_id = 1
for id in sorted_member_id:
  member_id_map[id] = start_id
  start_id = start_id + 1

For every message/notifcation connection, we will add 1 to the cell, and the normalise it.

# Inline Notifications

In [ ]:
orig_inline_notifications = pd.read_csv('/content/drive/MyDrive/text gcn/Hackforum data/CSV Data IronMarch/data_csv/orig_inline_notifications.csv')

We create a directed graph where the arrow points from "notify_from_id" to "notify_to_id"

In [ ]:
adjMatrix = np.zeros((orig_members.shape[0], orig_members.shape[0]))

Each message from A to B is counted separately. So if A and B have exchanged 3 notifications, then there would be 3 rows in the table, and hence the edge weight would be 3.

In [ ]:
for index, row in orig_inline_notifications.iterrows():
  from_id = row['notify_from_id']
  from_mapping = member_id_map[from_id]
  to_id = row['notify_to_id']
  to_mapping = member_id_map[to_id]
  # print(from_mapping, to_mapping)
  adjMatrix[from_mapping-1][to_mapping-1] +=1

## Sparsity of AdjMatrix

In [ ]:
countZero = adjMatrix[np.where(adjMatrix == 0)].size
countTotal = adjMatrix.shape[0]*adjMatrix.shape[1]
countZero/countTotal

0.9891251509441417

In [ ]:
np.save("/content/drive/MyDrive/text gcn/adjMatrix/ironmarch/adjMatrix_direct_notifs", adjMatrix)

# Profile Friends

In [ ]:
adjMatrix = np.zeros((orig_members.shape[0], orig_members.shape[0]))

In [ ]:
orig_profile_friends = pd.read_csv('/content/drive/MyDrive/text gcn/Hackforum data/CSV Data IronMarch/data_csv/orig_profile_friends.csv')

In [ ]:
for index, row in orig_profile_friends.iterrows():
  from_id = row['friends_member_id']
  to_id = row['friends_friend_id']
  if to_id not in member_id_map or from_id not in member_id_map:
    continue
  from_mapping = member_id_map[from_id]
  to_mapping = member_id_map[to_id]
  """-1 in the indices because we started the mapping from 1, and the matrix begins from 0."""
  adjMatrix[from_mapping-1][to_mapping-1] +=1

## Sparsity of the direct Edge connection matrix

In [ ]:
countZero = adjMatrix[np.where(adjMatrix == 0)].size
countTotal = adjMatrix.shape[0]*adjMatrix.shape[1]
countZero/countTotal

0.9921311509200936

In [ ]:
np.save("/content/drive/MyDrive/text gcn/adjMatrix/ironmarch/adjMatrix_direct_friends", adjMatrix)